In [1]:
import torch as th
x = th.tensor([1,2,3,4,5])

In [2]:
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


In [3]:
import syft as sy
hook = sy.TorchHook(th)

W0802 22:22:06.943528 26192 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\User\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0802 22:22:07.326839 26192 deprecation_wrapper.py:119] From C:\Users\User\Anaconda3\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Syft is an extension of torch.

In [4]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

Now we need to create a hook. Where we will pass the pytorch through the syft function and behind the scene it will modify some pytorch api so that we can use the torch and do what we need to do with privacy. 

In [5]:
bob = sy.VirtualWorker(hook, id='bob')
bob._objects

{}

In [6]:
bob = sy.VirtualWorker(hook, id='bob')
bob._objects
x = th.tensor([1,2,3,4,5])

In [7]:
x = x.send(bob)

In [8]:
bob._objects

{96946453345: tensor([1, 2, 3, 4, 5])}

In [9]:
x.location == bob

True

In [10]:
x.location

<VirtualWorker id:bob #objects:1>

In [11]:
x.id_at_location

96946453345

In [12]:
x.id

28024295088

In [13]:
x.owner

<VirtualWorker id:me #objects:0>

In [14]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [15]:
x

(Wrapper)>[PointerTensor | me:28024295088 -> bob:96946453345]

In [16]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [17]:
bob._objects

{}

In [18]:
#sending the tensor to multiple people

In [19]:
alice = sy.VirtualWorker(hook, id="alice")

In [20]:
x = th.tensor([1,2,3,4,5])

In [21]:
x_ptr = x.send(bob, alice)

In [22]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:91143492396 -> bob:2260759712]
	-> (Wrapper)>[PointerTensor | me:36424993486 -> alice:74961081706]

In [23]:
x_ptr.child

MultiPointerTensor>{'bob': (Wrapper)>[PointerTensor | me:91143492396 -> bob:2260759712], 'alice': (Wrapper)>[PointerTensor | me:36424993486 -> alice:74961081706]}

In [24]:
x_ptr.child.child

{'bob': (Wrapper)>[PointerTensor | me:91143492396 -> bob:2260759712],
 'alice': (Wrapper)>[PointerTensor | me:36424993486 -> alice:74961081706]}

In [25]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [26]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [27]:
x.get(sum_results = True)

tensor([ 2,  4,  6,  8, 10])

Introducing remote arithmatic 

In [28]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,2,3,4,5]).send(bob)

In [29]:
x

(Wrapper)>[PointerTensor | me:16537003907 -> bob:13539033117]

In [30]:
y

(Wrapper)>[PointerTensor | me:89057330367 -> bob:50634595427]

In [31]:
z = x + y

In [32]:
z

(Wrapper)>[PointerTensor | me:68075819821 -> bob:23698868054]

In [33]:
z = z.get()

In [34]:
z

tensor([ 2,  4,  6,  8, 10])

In [35]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:89801723473 -> bob:11817782090]

In [36]:
z = z.get()
z

tensor([ 2,  4,  6,  8, 10])

In [37]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)

In [38]:
z = (x + y).sum()

In [39]:
z.backward()

(Wrapper)>[PointerTensor | me:93082192468 -> bob:23116594075]

In [40]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [41]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [42]:
import numpy as np
np.array([[1,0,0,1,1]]).shape

(1, 5)

Learn a Simple Linear Model

In [43]:
input = th.tensor([[1.,1], [0,1], [1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [44]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [45]:
pred = input.mm(weights)

In [46]:
pred

(Wrapper)>[PointerTensor | me:43763742691 -> bob:10192295871]

In [47]:
loss=((pred - target)**2).sum()

In [48]:
loss.backward()
weights.data.sub_(weights.grad*0.1)
weights.grad *= 0

In [49]:
for i in range(10):
    pred = input.mm(weights)
    loss=((pred - target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad *= 0
    print(loss.get().data)

tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)


Garbage collection and common errors

In [50]:
bob = bob.clear_objects()

In [51]:
bob._objects

{}

In [52]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{7720178785: tensor([1, 2, 3, 4, 5])}

Now what will happen if want to delete this tensor. PySyft always assumes if you created a tensor and send it to someone, you should keep controlling the life cycle of that tensor. That means if you delete the pointer that you created the tensor that you are pointing to should also be deleted. 

In [54]:
bob._objects

{7720178785: tensor([1, 2, 3, 4, 5])}

In [55]:
x = 'asdf'

Now that we changed the value of x or I deleted the pointer that was pointing to bob, bob will loose the tensor. Let's check

In [56]:
bob._objects

{}

In [60]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [61]:
x

(Wrapper)>[PointerTensor | me:92680374126 -> bob:88967757821]

In [62]:
bob._objects

{88967757821: tensor([1, 2, 3, 4, 5])}

In [63]:
x = 'asdf'

In [64]:
bob._objects

{88967757821: tensor([1, 2, 3, 4, 5])}

Strangely there is still a reference to it. They never got gerbage collected. Even if you delete x, there will still be a pointer to bob.

In [65]:
del x

In [66]:
bob._objects

{88967757821: tensor([1, 2, 3, 4, 5])}

But it's not because an issue with PySyft

In [67]:
bob = bob.clear_objects()
bob._objects

{}

if we assign a tensor to bob thousand times. 

In [68]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [69]:
bob._objects

{72840841276: tensor([1, 2, 3, 4, 5])}

Look there is still only one tensor even after assigning x thousand times. It's because it reassigns x every time this loop runs and delete the x that already returning from there. 
Now why it is important. If you are doing federated learning or the exercise that we have done where we printed loss 10 times, they are also keep deleting preds and loss's in between. Otherwise it will generate thousands of tensors when we are doing forward and back propagation. 

In [70]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

If we add x and y it will give an error. because we are sending x to bob. So it's in another machine but y is in this machine. So, they cannot add together. 

In [71]:
#Toy federated learning

In [72]:
from torch import nn, optim

In [78]:
data = th.tensor([[1.,1], [0,1], [1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True)

In [79]:
#A Toy Model
model = nn.Linear(2,1)

In [80]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [81]:
opt.zero_grad()

In [82]:
pred = model(data)

In [83]:
loss = ((pred - target)**2).sum()

In [84]:
loss.backward()

In [85]:
opt.step()

In [87]:
def train():
    for iter in range(20):
        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        opt.step()
        print(loss.data)
train()

tensor(8.0663e-05)
tensor(5.7943e-05)
tensor(4.1915e-05)
tensor(3.0520e-05)
tensor(2.2358e-05)
tensor(1.6469e-05)
tensor(1.2192e-05)
tensor(9.0662e-06)
tensor(6.7683e-06)
tensor(5.0703e-06)
tensor(3.8098e-06)
tensor(2.8701e-06)
tensor(2.1671e-06)
tensor(1.6395e-06)
tensor(1.2423e-06)
tensor(9.4275e-07)
tensor(7.1634e-07)
tensor(5.4477e-07)
tensor(4.1468e-07)
tensor(3.1590e-07)


So, waht we will do now, we will send this data to other machines to train them in those machines. First we will split them and send them to two different machines

In [88]:
bob

<VirtualWorker id:bob #objects:2>

In [89]:
alice

<VirtualWorker id:alice #objects:0>

In [90]:
data_bob = data[0:2].send(bob)
target_bob = data[0:2].send(bob)

In [91]:
data_alice = data[0:2].send(alice)
target_alice = data[0:2].send(alice)

Let's make it a list of tuples

In [92]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [93]:
model = nn.Linear(2,1)

In [94]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [95]:
_data, _target = datasets[0]

In [96]:
_data

(Wrapper)>[PointerTensor | me:94964007359 -> bob:24210718882]

So, it's in bob's machine. 

In [97]:
_data.location

<VirtualWorker id:bob #objects:4>

All the data that we send have virtual workers. we need to send the model to those location now. 

In [98]:
model = model.send(_data.location)

This model has weight tensor and bias tensor. Every tensor has model.parameters. 

In [99]:
list(model.parameters())

[Parameter containing:
 Parameter>[PointerTensor | me:69225208499 -> bob:60940089572],
 Parameter containing:
 Parameter>[PointerTensor | me:77867010042 -> bob:74935475192]]

In [100]:
opt.zero_grad()

In [101]:
pred = model(_data)

In [103]:
loss = ((pred-_target)**2).sum()

In [104]:
loss.backward()

(Wrapper)>[PointerTensor | me:62246483649 -> bob:90184577596]

In [106]:
opt.step()  #brings the model back to us

In [107]:
model = model.get()

In [108]:
print(loss.get())

tensor(0.5305, requires_grad=True)


In [109]:
#Now put it all together
def train(iterations=20):
    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):
        for _data, _target in datasets:
            #send model to the data
            model = model.send(_data.location)
            #do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred-_target)**2).sum()
            loss.backward()
            opt.step()
            model = model.get()
            print(loss.get())

In [110]:
train()

tensor(6.3081, requires_grad=True)
tensor(4.4495, requires_grad=True)
tensor(3.1857, requires_grad=True)
tensor(2.3263, requires_grad=True)
tensor(1.7419, requires_grad=True)
tensor(1.3445, requires_grad=True)
tensor(1.0742, requires_grad=True)
tensor(0.8905, requires_grad=True)
tensor(0.7655, requires_grad=True)
tensor(0.6806, requires_grad=True)
tensor(0.6228, requires_grad=True)
tensor(0.5835, requires_grad=True)
tensor(0.5568, requires_grad=True)
tensor(0.5386, requires_grad=True)
tensor(0.5263, requires_grad=True)
tensor(0.5179, requires_grad=True)
tensor(0.5121, requires_grad=True)
tensor(0.5083, requires_grad=True)
tensor(0.5056, requires_grad=True)
tensor(0.5038, requires_grad=True)
tensor(0.5026, requires_grad=True)
tensor(0.5018, requires_grad=True)
tensor(0.5012, requires_grad=True)
tensor(0.5008, requires_grad=True)
tensor(0.5006, requires_grad=True)
tensor(0.5004, requires_grad=True)
tensor(0.5003, requires_grad=True)
tensor(0.5002, requires_grad=True)
tensor(0.5001, requi

Looks like this is a good way to privacy preservation. But still there is some problem. See, the model we send and the model we get back will be different. So, by looking at the difference we can somehow guess what that model does in the other machine. 
There are to ways to mitigate this. One way is we train the data with several iterations and with large amount of data. It gives bob some more privacy. Because it will be hard to reverse engineer. 
The other way is to send the model to train in seevral different machines simultaneously get bach the average of all the models. So the model we get back is the average model. So it will be really hard to know which machine did what change. 

In [111]:
#Advanced Remote Execution

In [112]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [113]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [114]:
x = th.tensor([1,2,3,4,5]).send(bob)
x

(Wrapper)>[PointerTensor | me:30775781119 -> bob:57281329751]

In [115]:
bob._objects

{57281329751: tensor([1, 2, 3, 4, 5])}

Now let's send this tensor to alice's machine

In [116]:
x = x.send(alice)

In [118]:
bob._objects

{57281329751: tensor([1, 2, 3, 4, 5])}

In [119]:
alice._objects

{30775781119: (Wrapper)>[PointerTensor | alice:30775781119 -> bob:57281329751]}

Bob still has that tensor and alice is now pointing to bob.

In [120]:
x

(Wrapper)>[PointerTensor | me:64729636153 -> alice:30775781119]

On the other hand, our machine is directly pointing to alice, not to bob anymore. So, if we send any message, it will go to alice's machine first and then from alice to bob. 

In [121]:
y = x+x
y

(Wrapper)>[PointerTensor | me:1640624099 -> alice:43621133571]

In [122]:
bob._objects

{57281329751: tensor([1, 2, 3, 4, 5]),
 71264602008: tensor([ 2,  4,  6,  8, 10])}

Bob had two tensors. One is x and the other is y

In [123]:
alice._objects

{30775781119: (Wrapper)>[PointerTensor | alice:30775781119 -> bob:57281329751],
 43621133571: (Wrapper)>[PointerTensor | alice:43621133571 -> bob:71264602008]}

Alice also has two tensors but both of them are pointers. That means the object x is pretty much owned by bob an alice to an extent that if we perform any action on it, it will be performed the same way in bob and alice's machine and we cannot do it solely in our machine. 

Let's create a new worker named jon

In [124]:
jon = sy.VirtualWorker(hook, id='jon')

In [147]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(jon)

Now we cannot add x and y because they are in two different machines. We can get the x back from alice's machine

In [148]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:99763734068 -> bob:68221462766]

In [149]:
bob._objects

{68221462766: tensor([1, 2, 3, 4, 5])}

In [151]:
alice._objects

{}

When we called x it came back from alice's machine but bob still has it. But is we call it again, it will be removed from bob's machine also

In [155]:
x = x.get()

In [156]:
bob._objects

{36046826009: tensor([1, 2, 3, 4, 5])}

In [153]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [142]:
bob._objects

{38315356642: tensor([1, 2, 3, 4, 5])}

In [143]:
alice._objects

{78712798017: (Wrapper)>[PointerTensor | alice:78712798017 -> bob:38315356642]}

In [144]:
del x

In [145]:
bob._objects

{}

In [146]:
alice._objects

{}

So all the garbage is collected. As soon as we deleted x it was deleted from bob and alice's machines

In [157]:
#Pointer chain operations

In [158]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [159]:
bob._objects

{15862662816: tensor([1, 2, 3, 4, 5])}

In [160]:
alice._objects

{45518932517: (Wrapper)>[PointerTensor | alice:45518932517 -> bob:15862662816]}

In [161]:
x.remote_get()

(Wrapper)>[PointerTensor | me:78869087403 -> alice:45518932517]

In [162]:
bob._objects

{}

In [163]:
alice._objects

{45518932517: tensor([1, 2, 3, 4, 5])}

In [164]:
x.move(bob)

(Wrapper)>[PointerTensor | me:78869087403 -> bob:78869087403]

Now x will move to bob from alice. 

In [165]:
x

(Wrapper)>[PointerTensor | me:78869087403 -> bob:78869087403]

In [166]:
bob._objects

{78869087403: tensor([1, 2, 3, 4, 5])}

In [167]:
alice._objects

{}

This is how we move the data from one worker to other.